In [5]:
# Célula 2: Geração do DataFrame de Testes (Versão Corrigida)
import pandas as pd
import numpy as np
from faker import Faker
import random
from datetime import datetime, timedelta

# Inicializa o Faker para o local Português-Brasil
fake = Faker('pt_BR')

def criar_base_prospects_hibrida(num_registros: int = 1000) -> pd.DataFrame:
    """
    Gera um DataFrame de prospects com uma mistura de Pessoas Físicas (PF)
    e Pessoas Jurídicas (PJ), contendo uma variedade de features relevantes
    para análise de crédito e recuperação de dívidas.
    """
    lista_de_prospects = []
    
    for i in range(num_registros):
        tipo_pessoa = random.choice(['PF', 'PJ'])
        
        # --- Dados Comuns a PF e PJ ---
        valor_divida_mil = round(np.random.lognormal(mean=3.0, sigma=1.5) * 10, 2)
        tempo_inadimplencia_dias = random.randint(30, 2000)
        
        prospect = {
            'id_cliente': 1000 + i,
            'valor_divida_mil': valor_divida_mil,
            'tempo_inadimplencia_dias': tempo_inadimplencia_dias,
            'email': fake.email(),
            'telefone': fake.phone_number(),
            'endereco': f"{fake.street_address()}, {fake.city()} - {fake.state_abbr()}",
            'possui_outras_dividas': random.choice(['SIM', 'NAO']),
            'possui_protestos': random.choice(['SIM', 'NAO']),
        }

        # --- Dados Específicos por Tipo de Pessoa ---
        if tipo_pessoa == 'PF':
            # CORREÇÃO APLICADA AQUI:
            # 1. Gerar a data de nascimento primeiro e guardá-la em uma variável.
            data_nascimento = fake.date_of_birth(minimum_age=18, maximum_age=80)
            # 2. Calcular a idade usando a variável.
            idade = (datetime.now().date() - data_nascimento).days // 365
            
            prospect.update({
                'documento': fake.cpf(),
                'nome_completo': fake.name(),
                # 3. Usar as variáveis pré-calculadas para atualizar o dicionário.
                'data_nascimento': data_nascimento,
                'idade': idade,
                'estado_civil': random.choice(['SOLTEIRO', 'CASADO', 'DIVORCIADO', 'VIUVO']),
                'profissao': fake.job(),
                'renda_mensal': round(np.random.lognormal(mean=8.0, sigma=0.8), 2),
                # Campos de PJ ficam nulos
                'razao_social': np.nan,
                'capital_social': np.nan,
                'faturamento_anual_mil': np.nan,
                'numero_funcionarios': np.nan,
                'ramo_atuacao_cliente': np.nan,
                'socios': np.nan,
            })
        else: # tipo_pessoa == 'PJ'
            socios = ", ".join([fake.name() for _ in range(random.randint(1, 4))])
            prospect.update({
                'documento': fake.cnpj(),
                'razao_social': fake.company(),
                'capital_social': round(random.uniform(50000, 5000000), 2),
                'faturamento_anual_mil': round(np.random.lognormal(mean=9, sigma=1.2) * 12, 2) / 1000,
                'numero_funcionarios': random.randint(2, 500),
                'ramo_atuacao_cliente': random.choice(['VAREJO', 'SERVICOS', 'INDUSTRIA', 'AGRONEGOCIO', 'TECNOLOGIA']),
                'socios': socios,
                # Campos de PF ficam nulos
                'nome_completo': np.nan,
                'data_nascimento': pd.NaT,
                'idade': np.nan,
                'estado_civil': np.nan,
                'profissao': np.nan,
                'renda_mensal': np.nan,
            })
            
        lista_de_prospects.append(prospect)

    df = pd.DataFrame(lista_de_prospects)
    
    # Reordenar colunas para melhor visualização
    colunas_pf = ['nome_completo', 'data_nascimento', 'idade', 'estado_civil', 'profissao', 'renda_mensal']
    colunas_pj = ['razao_social', 'capital_social', 'faturamento_anual_mil', 'numero_funcionarios', 'ramo_atuacao_cliente', 'socios']
    colunas_comuns = [col for col in df.columns if col not in colunas_pf and col not in colunas_pj]
    
    ordem_final = colunas_comuns[:2] + colunas_pf + colunas_pj + colunas_comuns[2:]
    
    return df[ordem_final]

# --- Geração e Análise da Base ---
print("Gerando base de dados sintética...")
df_prospects_teste = criar_base_prospects_hibrida(num_registros=2700)
print("Base de dados gerada com sucesso!")

# Salva o arquivo para ser usado pelo seu aplicativo Streamlit ou outros scripts
df_prospects_teste.to_csv('base_full.csv', index=False)
print("Arquivo 'base_full.csv' salvo.")

# Exibe informações sobre o DataFrame gerado
print("\n--- Informações da Base Gerada ---")
print(f"Total de registros: {len(df_prospects_teste)}")
print("\nTipos de dados por coluna:")
df_prospects_teste.info()

print("\n--- Amostra dos Dados (5 primeiros registros) ---")
display(df_prospects_teste.head())

Gerando base de dados sintética...
Base de dados gerada com sucesso!
Arquivo 'base_full.csv' salvo.

--- Informações da Base Gerada ---
Total de registros: 2700

Tipos de dados por coluna:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2700 entries, 0 to 2699
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id_cliente                2700 non-null   int64  
 1   valor_divida_mil          2700 non-null   float64
 2   nome_completo             1360 non-null   object 
 3   data_nascimento           1360 non-null   object 
 4   idade                     1360 non-null   float64
 5   estado_civil              1360 non-null   object 
 6   profissao                 1360 non-null   object 
 7   renda_mensal              1360 non-null   float64
 8   razao_social              1340 non-null   object 
 9   capital_social            1340 non-null   float64
 10  faturamento_anual_mil     1340 non-null

,id_cliente,valor_divida_mil,nome_completo,data_nascimento,idade,estado_civil,profissao,renda_mensal,razao_social,capital_social,...,numero_funcionarios,ramo_atuacao_cliente,socios,tempo_inadimplencia_dias,email,telefone,endereco,possui_outras_dividas,possui_protestos,documento
0,1000,43.39,Brayan Cirino,1992-08-23,33.0,CASADO,Tecnólogo em produção de bebidas,9818.39,NaN,NaN,...,NaN,NaN,NaN,346,amoreira@example.net,0500 594 0921,"Lago João Lucas da Conceição, 8, Mendonça - RJ",SIM,SIM,835.729.410-32
1,1001,178.74,NaN,NaT,NaN,NaN,NaN,NaN,Andrade Macedo e Filhos,2791837.12,...,244.0,TECNOLOGIA,"Isabela Costela, Luigi Fogaça, Isaac Azevedo",1824,liam88@example.net,+55 (071) 7322 5746,"Chácara de Cardoso, 60, Nunes - PI",NAO,SIM,39.078.426/0001-01
2,1002,7.11,NaN,NaT,NaN,NaN,NaN,NaN,Araújo S.A.,2430378.80,...,264.0,INDUSTRIA,Luiz Felipe Caldeira,1332,clarice03@example.org,+55 (011) 5669-3675,"Recanto Machado, da Luz das Flores - MS",NAO,SIM,68.021.573/0001-35
3,1003,236.92,NaN,NaT,NaN,NaN,NaN,NaN,Mendonça Ltda.,4280348.01,...,442.0,VAREJO,"Gael da Mata, Sofia da Rocha",412,isaaccasa-grande@example.net,+55 (071) 9610-3392,"Colônia Luísa Porto, 70, Macedo de Cavalcante ...",NAO,NAO,30.891.672/0001-03
4,1004,525.20,NaN,NaT,NaN,NaN,NaN,NaN,Moreira - ME,2048824.42,...,315.0,VAREJO,"Rebeca Barbosa, Kamilly Cunha",587,nicolenunes@example.net,(061) 3058-5973,"Residencial Lima, 15, Cavalcanti - TO",NAO,SIM,06.918.742/0001-87
